In [ ]:
def permut(qc,rounds):
    
    for r in range(rounds):
        for i in range(n):
            qc.cx(k[((n*r+i)%klen)],q[(0+i)%n])
            qc.cx(q[(47+i)%n],q[(0+i)%n])
            qc.ccx(q[(70+i)%n],q[(85+i)%n],anc[0])
            qc.cx(anc[0],q[(0+i)%n])
            qc.ccx(q[(70+i)%n],q[(85+i)%n],anc[0])
            qc.cx(q[(91+i)%n],q[(0+i)%n])
        
        
def keyinit(qc):
    permut(qc,r1)

def nonceinit(qc,nonce):
    qc.x(q[36])
    permut(qc,r2)
    for i in range(32):
        if(nonce[i]==1):
            qc.x(q[i+96])
    
    qc.x(q[36])
    permut(qc,r2)
    for i in range(32):
        if(nonce[i+32]==1):
            qc.x(q[i+96])
                
    qc.x(q[36])
    permut(qc,r2)
    for i in range(32):
        if(nonce[i+64]==1):
            qc.x(q[i+96])
            
def assodata(qc):
    qc.x(q[36])
    qc.x(q[37])
    permut(qc,r2)
    for i in range(32):
        if(ad[i]==1):
            qc.x(q[96+i])
            
    qc.x(q[36])
    qc.x(q[37])
    permut(qc,r2)
    for i in range(32):
        if(ad[i+32]==1):
            qc.x(q[96+i])
            
    qc.x(q[36])
    qc.x(q[37])
    permut(qc,r2)
    for i in range(32):
        if(ad[i+64]==1):
            qc.x(q[96+i])
    
def encrypt(qc):
    for j in range(int(n/32)):
        qc.x(q[36])
        qc.x(q[38])
        permut(qc,r1)
        for i in range(32):
            if(pt[32*j+i]==1):
                qc.x(q[96+i])
        for i in range(32):
            qc.cx(q[64+i],ct[32*j+i])
        for i in range(32):
            if(pt[32*j+i]==1):
                qc.x(ct[32*j+i])
    
n = 128

if(version == 128):
    klen = 128
    r1 = 9 # 9*128 = 1152
    r2 = 5 # 3*128 = 384, 5*128=640
if(version == 192):
    klen = 192
    r1 = 8 # 8*128 = 1024
    r2 = 5 # 3*128 = 384, 5*128=640
if(version == 256):
    klen = 256
    r1 = 10 # 10*128 = 1280
    r2 = 5 # 3*128 = 384, 5*128=640

from qiskit import *
%matplotlib inline
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.visualization import plot_histogram

q = QuantumRegister(n, name='q')
k = QuantumRegister(klen, name='k')
ct = QuantumRegister(n, name = 'ct')
anc = QuantumRegister(1, name='anc')

qc = QuantumCircuit(q,k,ct,anc)
qc.x(anc[0])

iv = [1]*96
ad = [1]*96
pt = [0]*128


keyinit(qc)
nonceinit(qc,iv)
assodata(qc)
encrypt(qc)